In [ ]:
%%capture
!pip install pennylane

In [2]:
import numpy as np
import cvxpy as cp
import scipy as sc
import time
import pennylane as qml
import torch
import torch.nn as nn
import torch.nn.functional as Fn
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd
import json

### 10 qubits

In [27]:
qubits = 10
num_wires = 2 * qubits
num_layers = qubits
N = 2 ** qubits
seed = 42
num_shots = 100

def claculate_entropy(rho):
    sigma = np.eye(N) / N
    np.seterr(divide='ignore', invalid='ignore')
    H_rho = np.real(np.trace(rho @ (sc.linalg.logm(rho) - sc.linalg.logm(sigma))))
    np.seterr(divide = 'warn', invalid='warn')
    return H_rho

def generate_random_circuit_structure(qubits, ratio_imprim=0.8, pauli_gates=['PauliX', 'PauliY', 'PauliZ'], seed=None):
    if seed:
        np.random.seed(seed)

    obj_wires = range(qubits)   
    num_qubits = len(obj_wires) * 2
    rng = np.random.default_rng(seed)

    num_gates = len(obj_wires) * len(pauli_gates)

    random_values = rng.random(3 * num_gates)
    random_choices = random_values[:num_gates] 
    gate_indices = (random_values[num_gates:2 * num_gates] * len(pauli_gates)).astype(int)
    wire_indices = (random_values[2 * num_gates:3 * num_gates] * len(obj_wires)).astype(int)
    cnot_wires = rng.choice(list(np.arange(num_qubits)), size=(num_gates, 2), replace=True)

    gate_choices = [pauli_gates[i] for i in gate_indices]
    wire_choices = [obj_wires[i] for i in wire_indices]

    circuit_structure = []

    for i in range(num_gates):
        if random_choices[i] < ratio_imprim:
            circuit_structure.append({"gate": "CNOT", "wires": list(cnot_wires[i])})
        else:
            circuit_structure.append({"gate": gate_choices[i], "wires": [wire_choices[i]]})
    
    return circuit_structure

def save_circuit_structure_to_json(circuit_structure, filename):
    with open(filename, 'w') as f:
        json.dump(circuit_structure, f)

def load_circuit_structure_from_json(filename):
    with open(filename, 'r') as f:
        circuit_structure = json.load(f)
    return circuit_structure

In [28]:
device = qml.device("default.qubit", wires=num_wires)
density_matrix = np.zeros((N, N), dtype=np.complex128)

@qml.qnode(device)
def measure_rho(param, circuit_structure, qubits, rotations=[qml.RX, qml.RY, qml.RZ]):
    obj_wires = range(qubits)   

    qml.Hadamard(wires=0)

    for gate_info in circuit_structure:
        gate = gate_info["gate"]
        wires = gate_info["wires"]
        if gate == "CNOT":
            qml.CNOT(wires=wires)
        elif gate == "PauliX":
            qml.PauliX(wires=wires[0])
        elif gate == "PauliY":
            qml.PauliY(wires=wires[0])
        elif gate == "PauliZ":
            qml.PauliZ(wires=wires[0])

    qml.RandomLayers(param, wires=obj_wires, rotations=rotations)

    result = qml.density_matrix(wires=obj_wires)
    return result


param = np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))
circuit_structure = generate_random_circuit_structure(qubits, seed=seed)
density_matrix = measure_rho(param, circuit_structure, qubits)
print(f"entropy: {claculate_entropy(density_matrix)}")

c:\Users\mitsu\Anaconda3\envs\qiskit_entropy\lib\site-packages\scipy\linalg\_matfuncs.py:200: LogmExactlySingularWarning: The logm input matrix is exactly singular.
  F = scipy.linalg._matfuncs_inv_ssq._logm(A)


entropy: 6.2383246250395


In [7]:
device = qml.device("default.qubit", wires=num_wires, shots=num_shots)
@qml.qnode(device)
def measure_rho(param, circuit_structure, qubits, rotations=[qml.RX, qml.RY, qml.RZ]):
    obj_wires = range(qubits)   

    qml.Hadamard(wires=0)

    for gate_info in circuit_structure:
        gate = gate_info["gate"]
        wires = gate_info["wires"]
        if gate == "CNOT":
            qml.CNOT(wires=wires)
        elif gate == "PauliX":
            qml.PauliX(wires=wires[0])
        elif gate == "PauliY":
            qml.PauliY(wires=wires[0])
        elif gate == "PauliZ":
            qml.PauliZ(wires=wires[0])

    qml.RandomLayers(param, wires=obj_wires, rotations=rotations)

    result = [qml.sample(qml.PauliZ(i)) for i in range(len(obj_wires))]
    return result

print(f"pauliz measurement: {measure_rho(param, circuit_structure, qubits)}")

pauliz measurement: [array([-1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,
        1., -1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1.,
        1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,  1., -1.,  1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.,
        1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,
        1., -1., -1.,  1.,  1., -1., -1.,  1., -1.]), array([-1., -1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,
       -1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,
        1.,  1.,  1.,  1., -1.,  1., -1., -1., -1., -1., -1.,  1.,  1.,
        1., -1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1.,  1., -1.,
       -1., -1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.,  1., -1.,
       -1., -1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.,  1.,

In [8]:
class neural_function(nn.Module):
    def __init__(self,dimension,hidden_layer):
        super(neural_function, self).__init__()

        self.dimension = dimension
        self.hidden_layer = hidden_layer
        self.lin1 = nn.Linear(self.dimension, self.hidden_layer)
        self.lin_end = nn.Linear(self.hidden_layer, 1)

    def forward(self, input):
        y = torch.sigmoid(self.lin1(input.float()))
        y = self.lin_end(y)

        return y

In [11]:
#@title Optimization using Gradient Descent (with neural network)

# parameters of the optimization
num_of_epochs = 3000
learning_rate = 0.05
num_of_samples = 100
dimension = int(num_wires/2)
hidden_layer = 2
np.random.seed(42)
batch_size = 100
num_batches = int(num_of_samples/batch_size)
 
# initialize the neural network and quantum circuit parameters
neural_fn = neural_function(dimension, hidden_layer)
param_init = np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))
# intialize the cost function store
cost_func_store = []

# start the training
for epoch in range(1, num_of_epochs):
    
  # evaluate the gradient with respect to the quantum circuit parameters
    gradients = np.zeros_like((param_init))
    
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):

      # copy the parameters
            shifted = param_init.copy()

      # right shift the parameters
            shifted[i, j] += np.pi/2

      # forward evaluation
            forward_sum = 0
            result = measure_rho(shifted, circuit_structure, qubits)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[q][sample] for q in range(dimension)])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                forward_sum += nn_result[0].detach().numpy()

      # normalize the forward sum
            forward_sum = forward_sum/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # backward evaluation
            backward_sum = 0
            result = measure_rho(shifted, circuit_structure, qubits)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[q][sample] for q in range(dimension)])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                backward_sum += nn_result[0].detach().numpy()

      # normalize the backward sum
            backward_sum = backward_sum/num_of_samples
      #print(backward_sum)

      # parameter-shift rule
            gradients[i, j] = - 0.5 * (forward_sum - backward_sum)
            
    np.save(f"./vn_net_gradients/gradients_epoch{epoch}_{time.time()}.npy", gradients)

  # first copy the quantum circuit parameters before updating it
    prev_param_init = param_init.copy()

  # update the quantum circuit parameters
    param_init -= learning_rate*gradients

  # evaluate the gradient with respect to the NN parameters
    optimizer = optim.SGD(neural_fn.parameters(), lr=learning_rate)

  # evaluate the first term
    loss = 0
    result = measure_rho(prev_param_init, circuit_structure, qubits)
    for sample in range(num_of_samples):
        # optimizer.zero_grad()
        sample_result_array = np.array([result[q][sample] for q in range(dimension)])
        random_result_array = np.random.choice([-1, 1], size=dimension)
        sample_nn_result = neural_fn(torch.from_numpy(sample_result_array))
        random_nn_result = neural_fn(torch.from_numpy(random_result_array))
        loss_term = (torch.exp(random_nn_result[0]) - sample_nn_result[0]).to("cpu")
        loss += loss_term / num_of_samples
    loss.backward()
    torch.nn.utils.clip_grad_norm_(neural_fn.parameters(), max_norm=1.0)
    optimizer.step()

  # evaluate the cost function at these parameters
    first_term = 0
    result = measure_rho(param_init, circuit_structure, qubits)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[q][sample] for q in range(dimension)])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        first_term += nn_result[0].detach().numpy()

  # normalize the cost sum
    first_term = first_term/num_of_samples

  # # Second term evaluation
    second_term = 0
    for sample in range(num_of_samples):
        result = np.random.choice([-1, 1], size=dimension)
        nn_result = neural_fn(torch.from_numpy(result.flatten()))
        second_term += np.exp(nn_result[0].detach().numpy())

  # normalize the second term sum
    second_term = second_term/num_of_samples

    # add the cost function to the store
    cost_func_store.append(np.log(N) - first_term + second_term - 1)

  # print the cost
    print(f"Epoch {epoch}: Loss: {loss.item()} Cost: {np.log(N) - first_term + second_term - 1}")

Epoch 1: Loss: 1.3725428581237793 Cost: 7.252277251210931
Epoch 2: Loss: 1.2783470153808594 Cost: 7.163819751170399
Epoch 3: Loss: 1.2549399137496948 Cost: 7.147150984135392
Epoch 4: Loss: 1.2114548683166504 Cost: 7.124427157071354
Epoch 5: Loss: 1.1878552436828613 Cost: 7.0745730063588645
Epoch 6: Loss: 1.1442835330963135 Cost: 7.051356744048121
Epoch 7: Loss: 1.1088011264801025 Cost: 7.037206482914211
Epoch 8: Loss: 1.0884407758712769 Cost: 7.000158559706929
Epoch 9: Loss: 1.0876907110214233 Cost: 6.991041922298194
Epoch 10: Loss: 1.0807983875274658 Cost: 6.972104674455645
Epoch 11: Loss: 1.045989751815796 Cost: 6.9627896642209315
Epoch 12: Loss: 1.0324633121490479 Cost: 6.9518276451052445
Epoch 13: Loss: 1.0336909294128418 Cost: 6.946937924948456
Epoch 14: Loss: 1.0183830261230469 Cost: 6.936047193509461
Epoch 15: Loss: 1.023155927658081 Cost: 6.9290831355274936
Epoch 16: Loss: 1.0004160404205322 Cost: 6.918740963068726
Epoch 17: Loss: 0.9930555820465088 Cost: 6.922340898727002
Epoc

MemoryError: Unable to allocate 4.00 MiB for an array with shape (2, 131072) and data type complex128

memory errorの発生対策でpure stateで回そうね、qubit(wire)数は2~16で頑張ろう